# Caption Analysis

In [2]:
import tiktoken
import pandas as pd
import openai
from time import sleep
import os
file_path = "../../openai_key.txt"

with open(file_path, "r") as file:
    key = file.read()
openai.api_key = key

In [3]:
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [4]:
# Word Counting Function
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [5]:
def get_questions_anwers(promt):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user",
         "content": promt}
    ])
    questions_answers = completion.choices[0].message.content
    return questions_answers

In [6]:
def chatgpt(promt):
    success = False
    while not success:
        try:
            qa_pairs = get_questions_anwers(promt)
            success = True
        except:
            print("retrying")
            sleep(5)
    return qa_pairs
    

In [7]:
def combined_promt(caption_1, caption_2, requirement):
    
    return f"{requirement}\ncontext 1: {caption_1} \ncontext 2: {caption_2}\noutput:"

def qa_generator(csv_path, demo = True):
    df = pd.read_csv(csv_path)
    if demo:
        df = df[:10]
    df = df[["id", "img", "caption_1", "caption_2"]]

    requirement = "You are required to ask about 10 questions about 2 given contexts. " + \
    "The question topics are: yes/no, counting, object, activity, " + \
    "color, posistion. Answer within 3 words.\n" + \
    "context 1: a man is jumping to catch a ball.\n" + \
    "context 2: a white ball is being catch by a man.\n" + \
    "output:1. What is he doing?\nJumping.\n2. What sport is he playing?\nFootball.\n3. What color is the ball?\nwhite."

    df['promt_3'] = df[["caption_1", "caption_2"]].apply(lambda row: combined_promt(row["caption_1"], row["caption_2"], requirement), axis=1)
    df['qa pairs'] = df['promt_3'].map(chatgpt)

    return df

result = qa_generator("clean_cartoon.csv")

retrying
retrying


In [13]:
result2 = result[["id", "img", "qa pairs"]]

In [15]:
def qa_processing(string):
    lines = string.split('\n')
    
    # Extract the question lines (odd indices)
    questions = [line.split('. ')[1] for line in lines[::2]]
    
    # Extract the answer lines (even indices)
    answers = [line for line in lines[1::2]]
    return questions, answers

In [31]:
annotation_data = []
question_data = []
answer_id = 1
question_id = 1
img_ids = []
imgs = []
questions = []
question_ids = []
answers = []
answer_ids = []
for idx in range(len(result2)):    
    img_id, img,  qa_pairs= result2.iloc[idx]
    ques, ans = qa_processing(qa_pairs)
    for i,q in enumerate(ques):
        ques_item = {
            'image_name': img,
            'image_id': img_id,
            'question': q,
            'question_id': question_id
        }
        question_data.append(ques_item)
        
        ann_item = {
            'question_type': q.split(" ")[0],
            'answers': ans[i],
            'image_id': img_id,
            'question_id': question_id,
            'answer_id': answer_id
        }

        annotation_data.append(ann_item )
        
        img_ids.append(img_id)
        imgs.append(img)
        questions.append(q)
        question_ids.append(question_id)
        answers.append(ans[i])
        answer_ids.append(answer_id)
        answer_id += 1
        question_id+=1
        


In [32]:
question_data[5:10]

[{'image_name': 'S28E11-07151',
  'image_id': 2,
  'question': 'What activity are the people doing?',
  'question_id': 6},
 {'image_name': 'S28E11-07151',
  'image_id': 2,
  'question': 'What is the position of the people?',
  'question_id': 7},
 {'image_name': 'S28E07-20101',
  'image_id': 3,
  'question': 'What is the man holding?',
  'question_id': 8},
 {'image_name': 'S28E07-20101',
  'image_id': 3,
  'question': 'What is he wearing?',
  'question_id': 9},
 {'image_name': 'S28E07-20101',
  'image_id': 3,
  'question': 'What else is on the wall?',
  'question_id': 10}]

In [33]:
annotation_data[5:10]

[{'question_type': 'What',
  'answers': 'Holding a cup.',
  'image_id': 2,
  'question_id': 6,
  'answer_id': 6},
 {'question_type': 'What',
  'answers': 'Standing.',
  'image_id': 2,
  'question_id': 7,
  'answer_id': 7},
 {'question_type': 'What',
  'answers': 'Stick.',
  'image_id': 3,
  'question_id': 8,
  'answer_id': 8},
 {'question_type': 'What',
  'answers': 'White shirt, blue pants.',
  'image_id': 3,
  'question_id': 9,
  'answer_id': 9},
 {'question_type': 'What',
  'answers': 'Mirror, clock.',
  'image_id': 3,
  'question_id': 10,
  'answer_id': 10}]

In [ ]:
data = pd.DataFrame([])
data["id"] = img_ids 
data["img"] = imgs
data["question id"] = question_ids
data["question"] = questions
data["answer id"] = answer_ids
answers = []
answer_ids = []
